<a href="https://colab.research.google.com/github/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/img/Banner_Bootcamp.png?raw=true">

#**Projeto Final de Conclusão de Curso**
###Bootcamp Data Science Aplicada 2 by [Alura](https://www.alura.com.br/) 
####Autor: Helton Cordeiro
e-mail: heltoncordeiro@gmail.com

Junho-Agosto/2021.



---

# **Objetivo desse notebook**

Após os dados terem sido disponibilizados pelo Hospital Sírio Libânes e tratado no notebook [projeto_final_tratamento_dados](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb) e termos trabalhando com alguns modelos de Machine Learning com seus valores default, vamos explorar os hiperparametros nesse notebook. 

Será utilizando um dicionário com o range de valores e parâmetros  que serão passados para o método **RandomizedSearchCV** utilizando a função *executa_modelos_RandomizedSearchCV* que está no arquivo funcoes.py.

A proposta da separação do [notebook de análise](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/Bootcamp_DataScience_projeto_final.ipynb) se deve ao tempo de processamento necessário para testar todos os modelos, além de facilitar futuros ajustes.

---


##O que é um hiperparâmetro?

Os hiperparâmetros contêm os dados que controlam o próprio processo de treinamento.

Seu aplicativo de treinamento lida com três categorias de dados durante o treinamento do modelo:

* Os *dados de entrada*, também chamados de dados de treinamento, formam uma coleção de registros individuais (instâncias) com as características que são importantes para o problema de machine learning. Esses dados configuram o modelo durante o treinamento para fazer predições precisas sobre novas instâncias de dados semelhantes. No entanto, os valores nos dados de entrada nunca se tornam diretamente parte do modelo.

* Os *parâmetros* do modelo são as variáveis que a técnica de machine learning escolhida usa para ajustar os dados. Por exemplo, uma rede neural profunda (DNN, na sigla em inglês) é composta por nós de processamento (neurônios), cada um com uma operação realizada nos dados enquanto eles trafegam pela rede. Quando a DNN é treinada, cada node tem um valor de peso que informa ao modelo o impacto que ele tem na predição final. Esses pesos são um exemplo dos parâmetros do modelo. De muitas formas, esses parâmetros são o modelo. Ou seja, são eles que diferenciam seu modelo específico de outros modelos do mesmo tipo que trabalham com dados semelhantes.

* Os *hiperparâmetros* são variáveis que controlam o próprio processo de treinamento. Por exemplo, faz parte da configuração de uma rede neural profunda decidir quantas camadas ocultas de nós precisam ser usadas entre a camada de entrada e a camada de saída, bem como quantos nós cada camada precisa usar. Essas variáveis não estão diretamente relacionadas aos dados de treinamento. Elas são variáveis de configuração. Os parâmetros mudam durante um job de treinamento, enquanto os hiperparâmetros geralmente permanecem constantes durante um job.


Os parâmetros do modelo são otimizados (ou seja, "ajustados") pelo processo de treinamento. Você executa os dados por meio das operações do modelo, compara a predição resultante com o valor real de cada instância de dados, avalia a precisão e ajusta até encontrar os melhores valores. Os hiperparâmetros são ajustados por meio da execução de todo o job de treinamento, a observação da precisão agregada e o ajuste. Nos dois casos, você está modificando a composição do modelo tentando encontrar a melhor combinação para lidar com o problema.

Fonte: [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)


##Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
from scipy.stats import randint

import warnings
warnings.simplefilter(action='ignore')

In [2]:
rstate = 73246
np.random.seed(rstate)

## Importando as funções auxiliares

* Arquivo funcoes.py contém as funções python que serão utilizados pelo projeto.

In [3]:
from funcoes import executa_modelos_RandomizedSearchCV

In [4]:
help(executa_modelos_RandomizedSearchCV)

Help on function executa_modelos_RandomizedSearchCV in module funcoes:

executa_modelos_RandomizedSearchCV(names, models, dados, n_splits, n_repeats, param_distributions, n_iter, showMsg=True)
    Função que recebe parâmetros para a execução de teste de hiperparametros dos modelos.
    Utiliza a função roda_modelo_RandomizedSearchCV()
    
    Parâmetros
    ----------
      names: array com os nomes dos modelos. É usado como o index do dataFrame de retorno.
               ex.: names = [ "KNeighbors", "Gaussian" ]
      models: array com a instância do modelo a ser testado.
               ex.: classes = [ KNeighborsClassifier(), 
                                GaussianProcessClassifier() ]
      dados: dataFrame com os dados
      n_splits: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      n_repeats: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      param_distributions: dicionário com parâmetros a serem testados
              ex: hiperparams = { 
          

## Carregando os dados
Vamos carregar o arquivo de dados tratados e sem as colunas com alta correção.

O processo de tratamento dos dados estão nesse notebook [projeto_final_tratamento_dados.ipynb](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb).


### Realizando a carga de dados

In [5]:
url='https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/dados/Kaggle_Sirio_Libanes_ICU_Prediction-tratado-sem-corr.xls?raw=true'
dados_raw = pd.read_excel(url, index_col=0)
dados = dados_raw.copy()



---


#Modelos de Machine Learning
Relação dos Modelos que serão testados nesse projeto, foram selecionados por resolverem problemas de classificação binária e aprendizado supervisionado.


1. [KNeighborsClassifier](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

2. [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

3. [GaussianProcessClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

4. [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

5. [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

6. [Neural Net (MLP)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

7. [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

8. [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

9. [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

10. [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


---



##Modelos

Criando as instâncias dos modelos para processamento.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

names = ["KNeighbors", "SVC", "Gaussian",
         "DecisionTree", "RandomForest", "NeuralMLP", "AdaBoost",
         "Gradient", "ExtraTrees", "LogisticRegression" ]

classes = [
    KNeighborsClassifier(),
    SVC(probability=True, random_state=rstate),
    GaussianProcessClassifier(random_state=rstate, warm_start=True),
    DecisionTreeClassifier(random_state=rstate),
    RandomForestClassifier(random_state=rstate, bootstrap=False, warm_start=True),
    MLPClassifier(random_state=rstate, warm_start=True),
    AdaBoostClassifier(random_state=rstate),
    GradientBoostingClassifier(random_state=rstate, warm_start=True),
    ExtraTreesClassifier(random_state=rstate, bootstrap=False, warm_start=True),
    LogisticRegression(random_state=rstate, warm_start=True) ]

##Testando Hiperparâmetros

Definindo o dicionário de parâmetros a serem testados.

In [7]:
hiperparams = {
    "KNeighbors" : {
        "n_neighbors" : randint(1, 25),
        "weights" : ["uniform", "distance"],
        "leaf_size" : randint(30, 300),
        "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"]
    },
    "SVC" : {
        "C": np.random.uniform(low=0.8, high=2, size=5),
        "kernel" : ["linear", "poly", "rbf", "sigmoid"],
        "gamma" : ["scale", "auto"],
        "shrinking": [True, False]
    },
    "Gaussian" : {
        "n_restarts_optimizer" : [0, 1, 2, 3],
        "max_iter_predict" : randint(100,500),
    },
    "DecisionTree" : {
        "criterion" : ["gini", "entropy"],
        "splitter": ["best", "random"],
        "max_depth" : randint(1, 100),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
        "max_features": ["sqrt", "log2"]
    },
    "RandomForest" : {
        "n_estimators" :randint(90, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(2, 100),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
        "max_features": ["sqrt", "log2"]
    },
    "NeuralMLP" : {
        "activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "alpha" : np.random.uniform(low=0.0001, high=0.01, size=10),
        "learning_rate" : ["constant", "invscaling", "adaptive"],
        "learning_rate_init" : np.random.uniform(low=0.001, high=0.1, size=10),
    },
    "AdaBoost" : {
        "n_estimators" : randint(50, 500),
        "learning_rate": np.random.uniform(low=0.8, high=5, size=10),
        "algorithm": ["SAMME", "SAMME.R"]
    },
    "Gradient" : {
        "loss": ["deviance", "exponential"],
        "learning_rate": np.random.uniform(low=1, high=2, size=10),
        "n_estimators" : randint(50, 500),
        "subsample":  np.random.uniform(low=0.1, high=1, size=10),
        "criterion" : ["friedman_mse", "mse"],
        "min_samples_split" : randint(1, 10),
        "min_samples_leaf" : randint(1, 10),
        "max_depth" : randint(2, 6)        
    },
    "ExtraTrees" : {
        "n_estimators" :randint(100, 1000),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 20),
        "min_samples_split" : randint(2, 100),
        "min_samples_leaf" : randint(1, 100),
    },
    "LogisticRegression" : {
        "C": np.random.uniform(low=0.1, high=2, size=10),
        "tol": np.random.uniform(low=0.0001, high=0.01, size=10),
        "solver"   : ["newton-cg", "lbfgs", "liblinear"], 
        "max_iter" : randint(90,500)
    }
}

## Processando

Vamos processar as modelos utilizando 5 splits de dados, com 10 repetições e 20 iterações.

In [8]:
dfmodelosRand = executa_modelos_RandomizedSearchCV(names, classes, dados, 5, 10, hiperparams, 20)

Modelo: KNeighbors 	 tempo: 12 segundos
Modelo: SVC 	 tempo: 36 segundos
Modelo: Gaussian 	 tempo: 25 segundos
Modelo: DecisionTree 	 tempo: 5 segundos
Modelo: RandomForest 	 tempo: 291 segundos
Modelo: NeuralMLP 	 tempo: 215 segundos
Modelo: AdaBoost 	 tempo: 465 segundos
Modelo: Gradient 	 tempo: 155 segundos
Modelo: ExtraTrees 	 tempo: 527 segundos
Modelo: LogisticRegression 	 tempo: 17 segundos


In [9]:
dfmodelosRand

,Modelo,AUC,Train AUC,Std AUC,Best Params,Tempo,objRandomizedSearchCV
Nome,,,,,,,
KNeighbors,"KNeighborsClassifier(algorithm='kd_tree', leaf...",0.700238,1.000000,0.061331,"{'algorithm': 'kd_tree', 'leaf_size': 292, 'n_...",12,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
SVC,"SVC(C=1.4048681047470488, break_ties=False, ca...",0.735924,0.851858,0.059675,"{'shrinking': False, 'kernel': 'linear', 'gamm...",36,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
Gaussian,"GaussianProcessClassifier(copy_X_train=True, k...",0.714948,0.950495,0.066657,"{'max_iter_predict': 199, 'n_restarts_optimize...",25,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
DecisionTree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.671283,0.757216,0.069798,"{'criterion': 'entropy', 'max_depth': 32, 'max...",5,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
RandomForest,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.771838,0.871363,0.063105,"{'criterion': 'entropy', 'max_depth': 79, 'max...",291,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
NeuralMLP,"MLPClassifier(activation='identity', alpha=0.0...",0.743558,0.837538,0.058227,"{'solver': 'sgd', 'learning_rate_init': 0.0338...",215,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
AdaBoost,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.717178,0.998454,0.063169,"{'algorithm': 'SAMME', 'learning_rate': 0.8438...",465,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
Gradient,"([DecisionTreeRegressor(ccp_alpha=0.0, criteri...",0.729589,1.000000,0.062214,"{'criterion': 'friedman_mse', 'learning_rate':...",155,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
ExtraTrees,"(ExtraTreeClassifier(ccp_alpha=0.0, class_weig...",0.755579,0.958914,0.068638,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",527,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...


Na coluna 'Modelo' temos o resultado do best_estimator_, e no campo 'objRandomizedSearchCV' temos uma cópia do objeto resultante do processamento, o retorno da função **RandomizedSearchCV**.

In [10]:
# O Tempo do processamento está em segundos.
dfmodelosRand['Tempo'].sum() / 60

29.133333333333333

# Resultado

Após o processamentos temos a lista dos melhores modelos e parâmetros de acordo com as configurações apresentadas no dicionário ```hiperparams```.

## ```best_params_```

In [11]:
for indice in names:
  print(100*'_')
  print(indice)
  print(20*'¨')
  display(dfmodelosRand.at[indice,'Best Params'])
print(100*'_')

____________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


{'algorithm': 'kd_tree',
 'leaf_size': 292,
 'n_neighbors': 24,
 'weights': 'distance'}

____________________________________________________________________________________________________
SVC
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


{'C': 1.4048681047470488,
 'gamma': 'auto',
 'kernel': 'linear',
 'shrinking': False}

____________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


...

____________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'n_estimators': 426}

____________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


{'C': 0.605274911004851,
 'max_iter': 227,
 'solver': 'liblinear',
 'tol': 0.003583528036845146}

____________________________________________________________________________________________________


## ```best_estimator_```

In [12]:
for indice in names:
  print(100*'_')
  print(indice)
  print(20*'¨')
  display(dfmodelosRand.at[indice,'Modelo'])
print(100*'_')

____________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


KNeighborsClassifier(algorithm='kd_tree', leaf_size=292, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=24, p=2,
                     weights='distance')

____________________________________________________________________________________________________
SVC
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


SVC(C=1.4048681047470488, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
    kernel='linear', max_iter=-1, probability=True, random_state=73246,
    shrinking=False, tol=0.001, verbose=False)

____________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


...

____________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=6, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=7,
                     min_weight_fraction_leaf=0.0, n_estimators=426,
                     n_jobs=None, oob_score=False, random_state=73246,
                     verbose=0, warm_start=True)

____________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


LogisticRegression(C=0.605274911004851, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=227, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=73246, solver='liblinear',
                   tol=0.003583528036845146, verbose=0, warm_start=True)

____________________________________________________________________________________________________


##params testados

lista de parâmetros pela ordem de teste.

In [13]:
for indice in names:
  obj = dfmodelosRand.at[indice,'objRandomizedSearchCV']
  dfParams = pd.DataFrame(obj.cv_results_['params']).reset_index()
  metricas = { 'rank_test_score': obj.cv_results_['rank_test_score'],
               'media_AUC_teste': obj.cv_results_['mean_test_score'],
               'media_AUC_treino':obj.cv_results_['mean_train_score'] }
  dfMetricas = pd.DataFrame(metricas).reset_index()
  dfAvaliacao = pd.merge(dfMetricas,dfParams, on="index")
  dfAvaliacao.drop(['index'],axis=1,inplace=True)
  print(200*'_')
  print(indice)
  print(20*'¨')
  display(dfAvaliacao)
print(200*'_')

________________________________________________________________________________________________________________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,algorithm,leaf_size,n_neighbors,weights
0,13,0.679854,0.821291,brute,133,7,uniform
1,6,0.693589,0.746585,ball_tree,299,22,uniform
2,2,0.695407,0.794489,kd_tree,254,11,uniform
3,17,0.602504,1.000000,kd_tree,175,2,distance
4,15,0.664438,1.000000,kd_tree,174,5,distance
5,1,0.700238,1.000000,kd_tree,292,24,distance
6,19,0.541158,1.000000,ball_tree,90,1,uniform
7,9,0.692173,0.753016,kd_tree,45,19,uniform
8,5,0.693686,0.800566,auto,160,10,uniform
9,6,0.693589,0.746585,ball_tree,93,22,uniform


________________________________________________________________________________________________________________________________________________________________________________________________________
SVC
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,shrinking,kernel,gamma,C
0,19,0.605388,0.627737,False,sigmoid,scale,1.404868
1,11,0.727165,0.782745,False,poly,auto,0.873736
2,7,0.727792,0.801420,False,poly,auto,1.561812
3,20,0.605387,0.627726,False,sigmoid,scale,0.873736
4,10,0.727332,0.812007,False,poly,scale,0.873736
5,8,0.727559,0.804731,False,poly,auto,1.712323
6,9,0.727531,0.785101,True,rbf,scale,1.404868
7,18,0.605438,0.627742,False,sigmoid,scale,1.712323
8,4,0.732463,0.853432,False,linear,scale,1.718927
9,13,0.722094,0.774277,True,rbf,scale,0.873736


________________________________________________________________________________________________________________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


...

________________________________________________________________________________________________________________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators
0,14,0.733607,0.772591,entropy,2,39,96,369
1,7,0.745627,0.857131,entropy,15,3,98,334
2,18,0.704946,0.717485,gini,3,83,19,965
3,3,0.748526,0.894405,entropy,15,3,62,496
4,2,0.749170,0.864877,gini,17,5,72,874
5,17,0.712438,0.728793,entropy,6,62,62,100
6,10,0.742684,0.812205,gini,16,19,90,535
7,16,0.730815,0.756298,entropy,4,50,98,230
8,6,0.745945,0.843548,entropy,7,10,65,729
9,5,0.747021,0.845909,gini,16,13,11,845


________________________________________________________________________________________________________________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,C,max_iter,solver,tol
0,6,0.747486,0.839885,0.767471,443,liblinear,0.005585
1,3,0.748872,0.838879,0.705739,476,newton-cg,0.000846
2,4,0.748392,0.839761,0.744853,428,lbfgs,0.007613
3,12,0.741723,0.852520,1.679049,476,newton-cg,0.006142
4,19,0.740534,0.852990,1.752223,350,liblinear,0.000846
5,20,0.739825,0.854655,1.968585,150,newton-cg,0.003584
6,8,0.747461,0.839870,0.767471,105,liblinear,0.009855
7,14,0.741546,0.852527,1.679049,395,newton-cg,0.009855
8,10,0.745907,0.804943,0.168018,256,lbfgs,0.009966
9,16,0.741217,0.853153,1.752223,393,newton-cg,0.005585


________________________________________________________________________________________________________________________________________________________________________________________________________


#Salvando nosso dataFrame

In [14]:
output = open('/content/drive/MyDrive/databases/dfmodelosHP', 'wb')
pickle.dump(dfmodelosRand, output)
output.close()

---

# Referências

* [Kaggle - COVID-19 - Clinical Data to assess diagnosis - Sírio Libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
* [SciKit Learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Configurar o treinamento do AutoML com Python](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-configure-auto-train)
* [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)
* [Sklearn.RandomizedSearchCV()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)